In [1]:
%load_ext autoreload
%autoreload 2
import logging
import sys, os
import talib
sys.path.insert(0, os.path.abspath(""+"../Crawler"))
sys.path.insert(0, os.path.abspath(""+"../Simulator"))

from Packing import Data_preprocessing
import Simulator
from Component import *

In [2]:
# Simulator 讀取DATA
sim = Simulator.Simulator()

# Update data
load origin pkl file.
load finance data pkl file.
Update origin data pkl.
    TX_Investors
    Institutional_investors
    Taifutures_LargeTrade
    MTX
    Stock_Investors
    TaiExchange_OHLC
    Taifutures_Investors
    MTX_Investors
    Stock_Price
    TaiExchange
    Taifutures
# Fill Nan. 
update finance data pkl.
Construct Components_lib.


In [25]:
# 大盤, Note: sim.DATA 可抓所有資料
TW_Futures = sim.DATA['加權指數']

# Buy signals
# talib_BBANDS 為舊寫法, 寫在Component.py
# Old: upperband, middleband, lowerband = talib_BBANDS(sim.DATA['加權指數'])
# 新寫法可以吃任何talib function, 也可控制技術指標的parameters
# New: 
talib_func_parameters = {'timeperiod':20, 'nbdevup':2, 'nbdevdn':2, 'matype':talib.MA_Type.T3 }
upper, middle, lower = talib_Output(sim.DATA['加權指數'], talib.abstract.BBANDS, talib_func_parameters = talib_func_parameters)

high_d  = Component(sim.DATA['加權指數']['High'])
open_d  = Component(sim.DATA['加權指數']['Open'])
close_d = Component(sim.DATA['加權指數']['Close'])
green_d = (open_d - close_d) > 30
buy = (upperband < high_d) & green_d

# Sell signals
sell = buy.shift(2)

# 4. Plot
# 灰色為此策略的持有時間
sim.Run(buy = buy, sell = sell, buy_price = 'Close')  
#sim.Iplotly(buy, sell, OHLC_data=TW_Futures, start_date='2016-01-01',end_date='2018-12-06')

INFO:Simulator:Mode : futures


# -----------------------RESULTS------------------------ #
#                                                        #
#               Period =      2011-09-01 to 2019-01-24   #
#         Num of Trade =              49                 #
#                Money =               1                 #
#               Profit =          -0.008                 #
#           Max Profit =           0.011                 #
#           Min Profit =           -0.03                 #
# Return On Investment =           -0.8%                 #
#       Annualized ROI =          -0.11%                 #
#        Ave_hold_days =             2.0                 #
#        Max_hold_days =               2                 #
#        Min_hold_days =               2                 #
#                                                        #
# ------------------------------------------------------ #

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [26]:
#sim.Setup(buy = buy, sell = sell).Iplotly(start_date='2016-01-01',end_date='2018-12-06')
sim.Iplotly(start_date='2016-01-01',end_date='2018-12-06')

INFO:Simulator:Mode : futures


# trade times: 20


In [27]:
# You can also re-plot the figure (add upperband & lowerband)
import plotly
import plotly.plotly as py

# Prepare data
start_date = '2017-01-01'
end_date = '2018-12-06'
OHLC_data = sim.DATA['加權指數'].truncate('2017-01-01', '2018-12-06', axis = 0)
upperband.data = upperband.data.truncate(OHLC_data.index[0], OHLC_data.index[-1], axis = 0)
lowerband.data = lowerband.data.truncate(OHLC_data.index[0], OHLC_data.index[-1], axis = 0)

# return fig only (return_iplot = False)
fig = sim.Iplotly(start_date=start_date,end_date=end_date, return_iplot = False)

# add upperBBAND & lowerBBAND
fig['data'].append( dict(
        type = 'scatter',
        y = upperband.data,
        x = upperband.data.index,
        yaxis='y1',
        name='UpperBAND'
    ) )
fig['data'].append( dict(
        type = 'scatter',
        y = lowerband.data,
        x = lowerband.data.index,
        yaxis='y1',
        name='LowerBAND'
    ) )

# Plot
py.iplot(fig, filename='TW_Futures')

INFO:Simulator:Mode : futures


# trade times: 14


In [28]:
# talib INFO
talib.abstract.BBANDS.info

{'display_name': 'Bollinger Bands',
 'function_flags': ['Output scale same as input'],
 'group': 'Overlap Studies',
 'input_names': OrderedDict([('price', 'close')]),
 'name': 'BBANDS',
 'output_flags': OrderedDict([('upperband',
               ['Values represent an upper limit']),
              ('middleband', ['Line']),
              ('lowerband', ['Values represent a lower limit'])]),
 'output_names': ['upperband', 'middleband', 'lowerband'],
 'parameters': OrderedDict([('timeperiod', 20),
              ('nbdevup', 2),
              ('nbdevdn', 2),
              ('matype', 8)])}

In [39]:
# Built-in Components 
# sim.Components. and press "tab"
# example:
散戶多空 = sim.Components.散戶多空()
散戶多空.data.head(5)

time
2016-01-04     7632
2016-01-05     9362
2016-01-06     9089
2016-01-07    11032
2016-01-08    11542
dtype: int64